In [3]:
# CDMA Multiplexing

import numpy as np

A = np.array([-1, -1, -1, 1, 1, -1, 1, 1])
B = np.array([-1, -1, 1, -1, 1, 1, 1, -1])
C = np.array([-1, 1, -1, 1, 1, 1, -1, -1])
D = np.array([-1, 1, -1, -1, -1, -1, 1, -1])

S1 = C
S2 = B + C
S3 = A + (-B)
S4 = A + (-B) + C
S5 = A + B + C + D
S6 = A + B + (-C) + D

print("S1 dot C = ", int(S1.dot(C)/8))
print("S2 dot C = ", int(S2.dot(C)/8))
print("S3 dot C = ", int(S3.dot(C)/8))
print("S4 dot C = ", int(S4.dot(C)/8))
print("S5 dot C = ", int(S5.dot(C)/8))
print("S6 dot C = ", int(S6.dot(C)/8))

S1 dot C =  1
S2 dot C =  1
S3 dot C =  0
S4 dot C =  1
S5 dot C =  1
S6 dot C =  -1


In [5]:
def compute_checksum(data):
    """
    Compute a simple checksum for the given string data.
    The checksum is defined as the sum of the ASCII values of the characters modulo 256.
    """
    return sum(ord(c) for c in data) % 256

class Frame:
    def __init__(self, seq, data):
        self.seq = seq
        self.data = data
        # The checksum is computed from the correct data.
        self.checksum = compute_checksum(data)
    
    def __str__(self):
        return f"Frame {self.seq}: Data='{self.data}', Checksum={self.checksum}"

def selective_repeat_simulation(frames, error_frames):
    """
    Simulate the Selective Repeat ARQ protocol.
    
    For each frame:
      - The frame is "sent" and its reception is simulated.
      - If the frame's sequence number is in error_frames, we simulate a corruption at the receiver
        by altering the data, so that the computed checksum does not match the transmitted checksum.
      - In that case, a NACK is issued and the frame is retransmitted.
    
    Parameters:
      frames: list of Frame objects to be transmitted.
      error_frames: set of sequence numbers for which the first transmission is to be corrupted.
    """
    print("\n=== Selective Repeat Protocol Simulation ===")
    
    for frame in frames:
        print("\n---------------------------")
        print(f"Transmitting {frame}")
        
        print(f"Frame {frame.seq}: Data sent -> [SENT]")
        
        # If this frame is designated to be corrupted on the first transmission:
        if frame.seq in error_frames:
            # Simulate corruption by modifying the data on reception.
            received_data = frame.data + "_CORRUPTED"
            computed_checksum = compute_checksum(received_data)
            if computed_checksum != frame.checksum:
                print(f"Frame {frame.seq}: Received corrupted. NACK sent.")
                # Retransmission (now sending the correct data)
                print(f"Frame {frame.seq}: Retransmitting...")
                print(f"Frame {frame.seq}: Data re-sent -> [RE-SENT]")
                # Now, the receiver gets the correct data.
                received_data = frame.data
                if compute_checksum(received_data) == frame.checksum:
                    print(f"Frame {frame.seq}: Received correctly. ACK sent.")
                else:
                    print(f"Frame {frame.seq}: Still corrupted. NACK sent.")
            else:
                print(f"Frame {frame.seq}: Received correctly. ACK sent.")
        else:
            # Normal transmission
            received_data = frame.data
            if compute_checksum(received_data) == frame.checksum:
                print(f"Frame {frame.seq}: Received correctly. ACK sent.")
            else:
                print(f"Frame {frame.seq}: Received corrupted. NACK sent.")

# ----------------------------
# Main: Run simulation with sample input/output.
# ----------------------------

if __name__ == "__main__":
 
    
    # --- Selective Repeat Protocol Simulation Sample ---
    # Create at least 7 frames with sequence numbers 0 to 6.
    frames = []
    for i in range(7):
        data = f"This is frame {i}"
        frame = Frame(seq=i, data=data)
        frames.append(frame)
    
    # Define frames that will experience an error on first transmission.
    # For example, simulate errors in frames 2 and 5.
    error_frames = {2, 5}
    
    print("\n>>> Selective Repeat Protocol Sample Input Frames:")
    for frame in frames:
        print(frame)
    
    print("\n>>> Selective Repeat Protocol Simulation Output:")
    selective_repeat_simulation(frames, error_frames)


>>> Selective Repeat Protocol Sample Input Frames:
Frame 0: Data='This is frame 0', Checksum=15
Frame 1: Data='This is frame 1', Checksum=16
Frame 2: Data='This is frame 2', Checksum=17
Frame 3: Data='This is frame 3', Checksum=18
Frame 4: Data='This is frame 4', Checksum=19
Frame 5: Data='This is frame 5', Checksum=20
Frame 6: Data='This is frame 6', Checksum=21

>>> Selective Repeat Protocol Simulation Output:

=== Selective Repeat Protocol Simulation ===

---------------------------
Transmitting Frame 0: Data='This is frame 0', Checksum=15
Frame 0: Data sent -> [SENT]
Frame 0: Received correctly. ACK sent.

---------------------------
Transmitting Frame 1: Data='This is frame 1', Checksum=16
Frame 1: Data sent -> [SENT]
Frame 1: Received correctly. ACK sent.

---------------------------
Transmitting Frame 2: Data='This is frame 2', Checksum=17
Frame 2: Data sent -> [SENT]
Frame 2: Received corrupted. NACK sent.
Frame 2: Retransmitting...
Frame 2: Data re-sent -> [RE-SENT]
Frame 2: 